# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
!git clone https://github.com/timesler/facenet-pytorch

Cloning into 'facenet-pytorch'...
remote: Enumerating objects: 1267, done.
remote: Total 1267 (delta 0), reused 0 (delta 0), pack-reused 1267
Receiving objects: 100% (1267/1267), 22.88 MiB | 1.80 MiB/s, done.
Resolving deltas: 100% (620/620), done.


In [2]:
!pip install -q facenet_pytorch

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
from PIL import Image
%cd /content/facenet-pytorch

/home/xps/anaconda3/envs/deeplearning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Errno 2] No such file or directory: '/content/facenet-pytorch'
/home/xps/projects/deep-learning-/CV/apps/Recognition/FaceVerify


#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [17]:
data_dir = 'facenet-pytorch/data/test_images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [2]:
dataset = datasets.ImageFolder("/home/xps/projects/deep-learning-/CV/apps/Recognition/FaceVerify/deploy/database/test_images_cropped")

In [5]:
dataset[0][1]

0

#### Determine if an nvidia GPU is available

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [97]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.7, post_process=True,
    device=device
)

In [98]:
img = Image.open("/home/xps/projects/deep-learning-/CV/apps/Recognition/FaceVerify/deploy/database/test_images/tranbaotri_20200083/5.jpg")

In [99]:
mtcnn(img, save_path='tests/1.jpg').shape
Image.open('tests/1.jpg')

AttributeError: 'NoneType' object has no attribute 'shape'

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [90]:
dataset = datasets.ImageFolder(data_dir)
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=1,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    if y[0].split('/')[-2][-1] == '3':
        print(y)
        print(x)
        img = x[0]
        print(mtcnn(x[0], save_path='facenet-pytorch/tests' + f'/{i}.jpg').shape)
        print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    

['facenet-pytorch/data/test_images_cropped/tranbaotri_20200083/1.jpg']
[<PIL.Image.Image image mode=RGB size=917x1920 at 0x7FDA6A3A9E80>]
torch.Size([3, 160, 160])
Batch 15 of 19['facenet-pytorch/data/test_images_cropped/tranbaotri_20200083/2.jpg']
[<PIL.Image.Image image mode=RGB size=917x1920 at 0x7FDA6A3A9F70>]
torch.Size([3, 160, 160])
Batch 16 of 19['facenet-pytorch/data/test_images_cropped/tranbaotri_20200083/3.jpg']
[<PIL.Image.Image image mode=RGB size=917x1920 at 0x7FDA6A536AF0>]
torch.Size([3, 160, 160])
Batch 17 of 19['facenet-pytorch/data/test_images_cropped/tranbaotri_20200083/4.jpg']
[<PIL.Image.Image image mode=RGB size=917x1920 at 0x7FDA6A3A9F10>]
torch.Size([3, 160, 160])
Batch 18 of 19['facenet-pytorch/data/test_images_cropped/tranbaotri_20200083/5.jpg']
[<PIL.Image.Image image mode=RGB size=917x1920 at 0x7FDA6A3A9E50>]
torch.Size([3, 160, 160])
Batch 19 of 19

In [93]:
transforms.ToTensor()(img).shape

torch.Size([3, 1920, 917])

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [9]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)
for params in resnet.parameters():
  params.requires_grad_(False)
for params in resnet.logits.parameters():
  params.requires_grad_(True)

#### Define optimizer, scheduler, dataset, and dataloader

In [12]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=32,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=32,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [14]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     1/1    | loss:    1.4733 | fps:    1.7771 | acc:    0.0000   

Epoch 1/8
----------
Train |     1/1    | loss:    1.2617 | fps:   12.3735 | acc:    0.3000   
Valid |     1/1    | loss:    1.1663 | fps:    3.3125 | acc:    0.0000   

Epoch 2/8
----------
Train |     1/1    | loss:    0.8988 | fps:   10.7183 | acc:    0.8000   
Valid |     1/1    | loss:    0.8966 | fps:    5.3609 | acc:    0.6667   

Epoch 3/8
----------
Train |     1/1    | loss:    0.7252 | fps:    9.6096 | acc:    0.9000   
Valid |     1/1    | loss:    0.6649 | fps:    5.0111 | acc:    1.0000   

Epoch 4/8
----------
Train |     1/1    | loss:    0.5130 | fps:    9.1453 | acc:    1.0000   
Valid |     1/1    | loss:    0.4826 | fps:    5.1708 | acc:    1.0000   

Epoch 5/8
----------
Train |     1/1    | loss:    0.3826 | fps:   12.7967 | acc:    1.0000   
Valid |     1/1    | loss:    0.3448 | fps:    7.6628 | acc:    1.0000   

Epoch 6/8
----------
Train |     1/1    | loss:    0.2

#test

In [117]:
!wget https://vcdn-ngoisao.vnecdn.net/2021/10/30/settopjolie-1635526336-6968-1635526893_1200x0.jpg -q -O test1.jpg 

In [118]:
from PIL import Image

In [141]:
test = mtcnn(Image.open("/content/facenet-pytorch/data/test_images/tranbaotri/6.jpg"), save_path='test1_mt.jpg').unsqueeze(0)
test = torch.concat([test, test], dim=0)

In [ ]:
resnet.eval()

In [143]:
torch.argmax(resnet(test))

tensor(3)

#Saving

In [149]:
!mkdir /content/facenet-pytorch/checkpoints

In [152]:
ckp_dir = "/content/facenet-pytorch/checkpoints"
torch.save(resnet.state_dict(), ckp_dir + 'model.pt')

#New person

In [127]:
!mkdir /content/facenet-pytorch/data/test_images/tranbaotri

#....


In [148]:
!git clone https://github.com/thangnch/MiAI_FaceRecog_2

Cloning into 'MiAI_FaceRecog_2'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 129 (delta 3), reused 9 (delta 0), pack-reused 114
Receiving objects: 100% (129/129), 2.70 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from flask import Flask
from flask import render_template , request
from flask_cors import CORS, cross_origin
import torchvision.transforms as tf
import argparse
import os
import sys
import math
import pickle
import align.detect_face
from PIL import Image
import numpy as np
import cv2
import collections
from sklearn.svm import SVC
import base64

FACENET_MODEL_PATH = '/content/facenet-pytorch/checkpoints' + 'model.pt'
mtcnn = MTCNN(keep_all=True, post_process=False)
#get from config
num_classes = 4
class_name = ['jolie', 'john', 'badd', 'tri']
class_dict = dict([(class_name[i], num_classes[i]) for i in range(num_classes)])

def get_face(image):
  mtcnn = MTCNN(keep_all=True, post_process=False)
  face = mtcnn(image)[0]
  image = transforms.Resize((160, 160))((transforms.ToTensor()((Image.fromarray(face.permute(1, 2, 0).numpy().astype(np.uint8))))))
  image = Image.fromarray(face.permute(1, 2, 0).numpy().astype(np.uint8))
  return image



# Load the model
print('Loading feature extraction model')
facenet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=num_classes
).to(device)

facenet.load_state_dict(torch.load(FACENET_MODEL_PATH))
facenet.eval()

# Get input and output tensors by POST



app = Flask(__name__)
CORS(app)



@app.route('/')
@cross_origin()
def index():
    return "OK!";

@app.route('/recog', methods=['POST'])
@cross_origin()
def upload_img_file():
    if request.method == 'POST':
        name = "Unknown"
        og_image = request.form.get('image')
        image = get_face(og_image)
        image = tf.ToTensor()(image)
        logits = facenet(image)
        name = class_dict[torch.argmax(logits).item()]


        

        return name;


if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0',port='8000')



In [ ]:

get_face()
